# Title
- working with different elements
    - buildings
        - fetching buildings from OSM
        - minor cleaning of buildings (geom type, valid geoms)
    - streets
        - fetching from OSM
        - preprocessing
    - tessellation
        - what and how
    - blocks
        - what and how

- measuring different things
    - dimensions
    - shapes
    - spatial distribution
    - intensity
    - connectivity
        - on nodes
        - on edges
    - diversity

- linking together

- understanding the context
- clustering
- maybe comparison if there's time

In [ ]:
import warnings

import libpysal
import momepy
import osmnx
import pandas

from clustergram import Clustergram

from bokeh.io import output_notebook
from bokeh.plotting import show

output_notebook()

warnings.filterwarnings('ignore', message='.*overflow encountered*')
warnings.filterwarnings('ignore', message='GeoSeries.isna')
warnings.filterwarnings('ignore', message='.*index_parts defaults to True')
warnings.filterwarnings('ignore', message='.*`op` parameter is deprecated*')

In [ ]:
place = 'Znojmo, Czechia'
local_crs = 5514

## Elements
### Buildings

In [ ]:
buildings = osmnx.geometries.geometries_from_place(place, tags={'building':True})

In [ ]:
buildings = buildings[["geometry"]].to_crs(local_crs)

In [ ]:
buildings.is_valid.all()

In [ ]:
buildings = buildings.explode(ignore_index=True)

In [ ]:
buildings["uID"] = range(len(buildings))

### Streets

In [ ]:
osm_graph = osmnx.graph_from_place(place, network_type='drive')
# TODO: check if 'drive' is the optimal type for the chosen case study

In [ ]:
osm_graph = osmnx.projection.project_graph(osm_graph, to_crs=local_crs)


In [ ]:
streets = osmnx.graph_to_gdfs(
    osm_graph, 
    nodes=False, 
    edges=True,
    node_geometry=False, 
    fill_edge_geometry=True
)

In [ ]:
streets = momepy.remove_false_nodes(streets)
streets = streets[["geometry"]]

### Tessellation

In [ ]:
limit = momepy.buffered_limit(buildings, 100)

tessellation = momepy.Tessellation(buildings, "uID", limit, verbose=False)
tessellation = tessellation.tessellation

### Link streets

In [ ]:
streets["nID"] = range(len(streets))
buildings['nID'] = momepy.get_network_id(buildings, streets, 'nID', min_size=300, verbose=False)
tessellation = tessellation.merge(buildings[['uID', 'nID']], on='uID', how='left')

## Measure

### Dimensions

In [ ]:
buildings["area"] = buildings.area
tessellation["area"] = tessellation.area
streets["length"] = streets.length

### Shape

In [ ]:
buildings['eri'] = momepy.EquivalentRectangularIndex(buildings).series

In [ ]:
buildings['elongation'] = momepy.Elongation(buildings).series
tessellation['convexity'] = momepy.Convexity(tessellation).series

In [ ]:
streets["linearity"] = momepy.Linearity(streets).series

### Spatial distribution

In [ ]:
buildings['stbOri'] = momepy.Orientation(buildings, verbose=False).series
 
tessellation['stcOri'] = momepy.Orientation(tessellation, verbose=False).series
buildings['stbCeA'] = momepy.CellAlignment(buildings, tessellation, 'stbOri', 'stcOri', 'uID', 'uID').series

In [ ]:
buildings["mtbSWR"] = momepy.SharedWallsRatio(buildings).series

In [ ]:
queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tessellation, ids="uID", silence_warnings=True)

In [ ]:
buildings["mtbAli"] = momepy.Alignment(buildings, queen_1, "uID", "stbOri", verbose=False).series
buildings["mtbNDi"] = momepy.NeighborDistance(buildings, queen_1, "uID", verbose=False).series
tessellation["mtcWNe"] = momepy.Neighbors(tessellation, queen_1, "uID", weighted=True, verbose=False).series
tessellation["mdcAre"] = momepy.CoveredArea(tessellation, queen_1, "uID", verbose=False).series

In [ ]:
buildings_q1 = libpysal.weights.contiguity.Queen.from_dataframe(buildings, silence_warnings=True)

In [ ]:
queen_3 = momepy.sw_high(k=3, weights=queen_1)

buildings['ltbIBD'] = momepy.MeanInterbuildingDistance(buildings, queen_1, 'uID', queen_3, verbose=False).series
buildings['ltcBuA'] = momepy.BuildingAdjacency(buildings, queen_3, 'uID', buildings_q1, verbose=False).series

In [ ]:
profile = momepy.StreetProfile(streets, buildings)
streets["width"] = profile.w
streets["width_deviation"] = profile.wd
streets["openness"] = profile.o

### Intensity

In [ ]:
tessellation['sicCAR'] = momepy.AreaRatio(tessellation, buildings, 'area', 'area', 'uID').series

### Connectivity

In [ ]:
graph = momepy.gdf_to_nx(streets)
graph = momepy.node_degree(graph)
graph = momepy.closeness_centrality(graph, radius=400, distance="mm_len")
graph = momepy.meshedness(graph, radius=400, distance="mm_len")

In [ ]:
nodes, streets = momepy.nx_to_gdf(graph)

In [ ]:
buildings["nodeID"] = momepy.get_node_id(buildings, nodes, streets, "nodeID", "nID")

In [ ]:
merged = tessellation.merge(buildings.drop(columns=['nID', 'geometry']), on='uID')
merged = merged.merge(streets.drop(columns='geometry'), on='nID', how='left')
merged = merged.merge(nodes.drop(columns='geometry'), on='nodeID', how='left')

In [ ]:
merged.columns

## Understanding the context

In [ ]:
percentiles = []
for column in merged.columns.drop(["uID", "nodeID", "nID", 'mm_len', 'node_start', 'node_end', "geometry"]):
    perc = momepy.Percentiles(merged, column, queen_3, "uID", verbose=False).frame
    perc.columns = [f"{column}_" + str(x) for x in perc.columns]
    percentiles.append(perc)

In [ ]:
percentiles_joined = pandas.concat(percentiles, axis=1)

In [ ]:
percentiles_joined

In [ ]:
standardized = (percentiles_joined - percentiles_joined.mean()) / percentiles_joined.std()

In [ ]:
standardized

In [ ]:
cgram = Clustergram(range(1, 12), n_init=10)
cgram.fit(standardized.fillna(0))

In [ ]:
show(cgram.bokeh())

In [ ]:
show(cgram.bokeh(pca_weighted=False))

In [ ]:
merged["cluster"] = cgram.labels[8].values

In [ ]:
buildings = buildings.drop(columns=[col for col in buildings.columns if "cluster" in col]).merge(merged[["uID", "cluster"]], on="uID")

In [ ]:
buildings.plot("cluster", categorical=True, figsize=(20, 20))

In [ ]:
buildings.explore("cluster", categorical=True, prefer_canvas=True, tiles="CartoDB Positron", tooltip=False)